In [28]:
# Let's import our dependencies. 
import pandas as pd
import requests
import json
import numpy as np

# Import the API key
from config import geoapify_key  

In [29]:
# Read our CSV file into pandas dataframe. 

miami_df = pd.read_csv('miami-housing_2016.csv')
miami_df.head()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4


In [53]:
miami_df['LATITUDE_scale']= np.round(miami_df['LATITUDE'],3)
miami_df['LONGITUDE_scale']= np.round(miami_df['LONGITUDE'],3)

In [54]:
miami_df['LATITUDE_scale'] = miami_df['LATITUDE_scale'].map('{:.3f}'.format)
miami_df['LONGITUDE_scale'] = miami_df['LONGITUDE_scale'].map('{:.3f}'.format)

In [57]:
# Check the dataframe to ensure that there are no missing values. 
miami_df.head(10)
#miami_df.info()
#miami_df['LATITUDE_scale'].unique()
#miami_df['LONGITUDE_scale'].nunique()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality,LATITUDE_scale,LONGITUDE_scale
0,25.891031,-80.160561,622280070620,440000.0,9375,1753,0,2815.9,12811.4,347.6,42815.3,37742.2,15954.9,67,0,8,4,25.891,-80.161
1,25.891324,-80.153968,622280100460,349000.0,9375,1715,0,4359.1,10648.4,337.8,43504.9,37340.5,18125.0,63,0,9,4,25.891,-80.154
2,25.891334,-80.153740,622280100470,800000.0,9375,2276,49206,4412.9,10574.1,297.1,43530.4,37328.7,18200.5,61,0,2,4,25.891,-80.154
3,25.891765,-80.152657,622280100530,988000.0,12450,2058,10033,4585.0,10156.5,0.0,43797.5,37423.2,18514.4,63,0,9,4,25.892,-80.153
4,25.891825,-80.154639,622280100200,755000.0,12800,1684,16681,4063.4,10836.8,326.6,43599.7,37550.8,17903.4,42,0,7,4,25.892,-80.155
5,25.892060,-80.161354,622280070180,630000.0,9900,1531,2978,2391.4,13017.0,188.9,43135.1,38176.2,15687.2,41,0,2,4,25.892,-80.161
6,25.892473,-80.157217,622280080100,1020000.0,10387,1753,23116,3277.4,11667.8,0.0,43598.7,37973.9,17068.2,63,0,2,5,25.892,-80.157
7,25.893019,-80.157426,622280080400,850000.0,10272,1663,34933,3112.4,11718.1,10.5,43780.8,38198.3,16989.9,21,0,9,4,25.893,-80.157
8,25.893046,-80.161556,622280080020,250000.0,9375,1493,11668,2081.8,13043.8,51.5,43481.7,38542.0,15623.3,56,0,3,4,25.893,-80.162
9,25.893050,-80.158048,622280080370,1220000.0,13803,3077,34580,2937.7,11917.7,9.7,43730.1,38235.2,16787.0,63,0,11,5,25.893,-80.158


In [63]:
check_lat_lon =[]
lat_list = []
long_list =[]
school_count =[]
zip_list =[]
for index,row in miami_df.iterrows():
    lat_lon = row["LATITUDE_scale"]+row["LONGITUDE_scale"]
    if lat_lon not in check_lat_lon:
        check_lat_lon.append(lat_lon)
        categories = "education.school"
        radius = 5000
        filters = f'circle:{row["LONGITUDE_scale"]},{row["LATITUDE_scale"]},{radius}'
        bias = f'proximity:{row["LONGITUDE_scale"]},{row["LATITUDE_scale"]}'
        limit = 100

        # set up a parameters dictionary
        params = {
            "categories":categories,
            "bias":bias,
            "apiKey":geoapify_key,    
            "limit":limit,
            "filter":filters,
            "bias":bias,
        }
        base_url = "https://api.geoapify.com/v2/places"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)   
        # print the response url, avoid doing for public github repos in order to avoid exposing key
        print(response.url)   
        lat_list.append(row["LATITUDE_scale"])
        long_list.append(row["LONGITUDE_scale"])
        places_data = response.json()
        school_count.append(len(places_data["features"]))
        zip_list.append(places_data["features"][0]["properties"]["postcode"])                        

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.161%2C25.891&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.161%2C25.891%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.154%2C25.891&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.154%2C25.891%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.153%2C25.892&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.153%2C25.892%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.155%2C25.892&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.155%2C25.892%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.161%2C25.892&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.161%2C25.892%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.140%2C25.935&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.140%2C25.935%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.139%2C25.937&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.139%2C25.937%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.123%2C25.948&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.123%2C25.948%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.124%2C25.949&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.124%2C25.949%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.123%2C25.949&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.123%2C25.949%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.172%2C25.896&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.172%2C25.896%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.173%2C25.896&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.173%2C25.896%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.175%2C25.896&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.175%2C25.896%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.173%2C25.897&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.173%2C25.897%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.179%2C25.897&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.179%2C25.897%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.186%2C25.907&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.186%2C25.907%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.184%2C25.907&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.184%2C25.907%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.181%2C25.908&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.181%2C25.908%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.178%2C25.908&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.178%2C25.908%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.175%2C25.908&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.175%2C25.908%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.173%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.173%2C25.916%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.169%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.169%2C25.916%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.170%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.170%2C25.916%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.183%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.183%2C25.916%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.184%2C25.916&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.184%2C25.916%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

...

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.236%2C25.914&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.236%2C25.914%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.217%2C25.914&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.217%2C25.914%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.242%2C25.914&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.242%2C25.914%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.241%2C25.914&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.241%2C25.914%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.229%2C25.915&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.229%2C25.915%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.250%2C25.903&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.250%2C25.903%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.257%2C25.904&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.257%2C25.904%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.256%2C25.904&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.256%2C25.904%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.251%2C25.905&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.251%2C25.905%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.252%2C25.907&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.252%2C25.907%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.248%2C25.923&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.248%2C25.923%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.268%2C25.923&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.268%2C25.923%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.249%2C25.924&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.249%2C25.924%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.268%2C25.924&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.268%2C25.924%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.251%2C25.924&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.251%2C25.924%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.274%2C25.856&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.274%2C25.856%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.264%2C25.856&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.264%2C25.856%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.278%2C25.856&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.278%2C25.856%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.280%2C25.857&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.280%2C25.857%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.265%2C25.857&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=100&filter=circle%3A-80.265%2C25.857%2C5000
https://api.geoapify.com/v2/places?categories=education.school&bi

KeyboardInterrupt: 

In [67]:
summary_df = pd.DataFrame( {
    "lat" : lat_list,
    "lon": long_list,
    "count" : school_count,
    "postcodes" : zip_list
    })

In [68]:
summary_df.head()

,lat,lon,count,postcodes
0,25.891,-80.161,97,33161
1,25.891,-80.154,90,33181
2,25.892,-80.153,86,33181
3,25.892,-80.155,91,33181
4,25.892,-80.161,95,33161


In [62]:
miami_df.describe()

,LATITUDE,LONGITUDE,PARCELNO,SALE_PRC,LND_SQFOOT,TOT_LVG_AREA,SPEC_FEAT_VAL,RAIL_DIST,OCEAN_DIST,WATER_DIST,CNTR_DIST,SUBCNTR_DI,HWY_DIST,age,avno60plus,month_sold,structure_quality
count,13932.000000,13932.000000,1.393200e+04,1.393200e+04,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000,13932.000000
mean,25.728811,-80.327475,2.356496e+12,3.999419e+05,8620.879917,2058.044574,9562.493468,8348.548715,31690.993798,11960.285235,68490.327132,41115.047265,7723.770693,30.669251,0.014930,6.655828,3.513997
std,0.140633,0.089199,1.199290e+12,3.172147e+05,6070.088742,813.538535,13890.967782,6178.027333,17595.079468,11932.992369,32008.474808,22161.825935,6068.936108,21.153068,0.121276,3.301523,1.097444
min,25.434333,-80.542172,1.020008e+11,7.200000e+04,1248.000000,854.000000,0.000000,10.500000,236.100000,0.000000,3825.600000,1462.800000,90.200000,0.000000,0.000000,1.000000,1.000000
25%,25.620056,-80.403278,1.079160e+12,2.350000e+05,5400.000000,1470.000000,810.000000,3299.450000,18079.350000,2675.850000,42823.100000,23996.250000,2998.125000,14.000000,0.000000,4.000000,2.000000
50%,25.731810,-80.338911,3.040300e+12,3.100000e+05,7500.000000,1877.500000,2765.500000,7106.300000,28541.750000,6922.600000,65852.400000,41109.900000,6159.750000,26.000000,0.000000,7.000000,4.000000
75%,25.852269,-80.258019,3.060170e+12,4.280000e+05,9126.250000,2471.000000,12352.250000,12102.600000,44310.650000,19200.000000,89358.325000,53949.375000,10854.200000,46.000000,0.000000,9.000000,4.000000
max,25.974382,-80.119746,3.660170e+12,2.650000e+06,57064.000000,6287.000000,175020.000000,29621.500000,75744.900000,50399.800000,159976.500000,110553.800000,48167.300000,96.000000,1.000000,12.000000,5.000000


In [7]:
# Determine if there are any duplicate parcel numbers. 

miami_df["PARCELNO"].nunique()

# We see there are duplicate parcel numbers, but it seems that this is because the same property was sold twice (or more) in the same year.

13776

In [21]:
# Set the geographical coordinates for Miami
latitude = 25.7823404
longitude = -80.3695441

# Set the parameters for the type of place
categories = "education.school"
radius = 5000

In [22]:
# Set the parameters for the type of search
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 100

# set up a parameters dictionary
params = {
    "categories":categories,
    "bias":bias,
    "apiKey":geoapify_key,    
    "limit":limit,
    "filter":filters,
    "bias":bias,
}

In [23]:

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# run a request using our params dictionary
response = requests.get(base_url, params=params)   
# print the response url, avoid doing for public github repos in order to avoid exposing key
print(response.url)  


https://api.geoapify.com/v2/places?categories=education.school&bias=proximity%3A-80.3695441%2C25.7823404&apiKey=b9b490ba1c374ffe82e78ee87bb6df2e&limit=20&filter=circle%3A-80.3695441%2C25.7823404%2C10000


In [24]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -80.37394189999999,
                    25.77287750018688
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Miami Christian School",
                "address_line2": "Northwest 109th Avenue, Lil Abner Mobile Home Park, FL 33199, United States of America",
                "categories": [
                    "education",
                    "education.school"
                ],
                "city": "Lil Abner Mobile Home Park",
                "country": "United States",
                "country_code": "us",
                "county": "Miami-Dade County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "amenity": "school",
                 

In [25]:
# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -80.37394189999999,
                    25.77287750018688
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Miami Christian School",
                "address_line2": "Northwest 109th Avenue, Lil Abner Mobile Home Park, FL 33199, United States of America",
                "categories": [
                    "education",
                    "education.school"
                ],
                "city": "Lil Abner Mobile Home Park",
                "country": "United States",
                "country_code": "us",
                "county": "Miami-Dade County",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "amenity": "school",
                 